## Implementation of K_NN

### 1.Converting Data Frame to Pytorch tensor

    - load encoding csv file into data frame
    - take only the encoded columns and convert into numpy
    - now convert those numpy values to pytorch tensor and copy to the GPU

In [1]:
import torch
from PIL import Image
from torchvision import transforms
import torchvision.models as models
import pandas as pd
import numpy as np
import os
import time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## kNN on Alex Model

In [3]:
df = pd.read_csv('data/alex_encoding/encoding_file.csv')

In [4]:
df

,id,image_id,encoded_0,encoded_1,encoded_2,encoded_3,encoded_4,encoded_5,encoded_6,encoded_7,...,encoded_1014,encoded_1015,encoded_1016,encoded_1017,encoded_1018,encoded_1019,encoded_1020,encoded_1021,encoded_1022,encoded_1023
0,000075604a,6bdca6e490.jpg,0.213690,0.254143,0.135755,0.871673,0.000000,0.173298,2.298700,0.127594,...,0.143134,1.333617,0.320830,1.209395,1.983253,1.801040,0.000000,0.000000,0.000000,0.000000
1,00007bfd16,6409eab844.jpg,3.736162,6.597008,4.941602,1.239078,0.403210,0.318469,0.222777,0.782325,...,0.500033,0.977747,0.000000,0.031324,0.000000,0.000000,0.033263,0.000000,0.025164,0.000000
2,000095fc1d,a1374cdd98.jpg,3.113306,2.854025,2.688789,1.358068,1.585247,2.266678,0.548850,0.971478,...,0.635072,0.196143,0.680662,0.336033,1.607502,1.960970,0.000000,0.000000,0.000000,0.005487
3,0000b1e2b5,cb1a684683.jpg,0.055429,0.000000,2.824535,0.000000,0.240313,0.059274,0.440367,0.014268,...,1.247056,1.347328,1.809847,0.238808,2.559675,0.465984,0.975320,1.021264,0.119180,0.146033
4,0000c79afb,2f4b4c4452.jpg,0.484349,1.256083,0.359304,1.275254,0.093606,0.042257,0.000000,0.000000,...,0.244099,0.451293,0.000000,0.000000,0.000000,0.000000,0.674627,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280742,ffff04dc41,30eb51d14c.jpg,1.013533,1.918724,1.680400,0.101543,0.133608,0.000000,3.744615,0.381688,...,0.183170,0.323274,1.467262,2.938093,0.254057,1.577386,0.024614,0.296276,0.000000,0.258263
280743,ffff43701e,eda1a4e605.jpg,4.995702,2.675699,0.902978,2.184537,3.057896,1.699590,5.906440,2.120128,...,0.857434,1.365958,0.000000,0.333333,0.016225,0.801414,0.000000,0.000000,0.000000,0.000000
280744,ffffcd4444,be3b1c7122.jpg,0.000000,0.000000,0.000000,0.000000,2.423423,1.015439,1.725580,0.000000,...,3.084663,1.221924,0.427211,1.516287,2.285784,4.832259,0.810528,0.000000,0.040364,0.509820
280745,ffffd33513,15c1a2afd4.jpg,0.502969,0.043087,0.008109,0.000000,0.189479,0.626572,1.119680,1.533680,...,0.098253,0.331980,4.437345,0.875123,0.000000,0.000000,0.073967,0.000000,0.103370,0.000000


In [4]:
df[df.columns[2:]]

,encoded_0,encoded_1,encoded_2,encoded_3,encoded_4,encoded_5,encoded_6,encoded_7,encoded_8,encoded_9,...,encoded_1014,encoded_1015,encoded_1016,encoded_1017,encoded_1018,encoded_1019,encoded_1020,encoded_1021,encoded_1022,encoded_1023
0,0.213690,0.254143,0.135755,0.871673,0.000000,0.173298,2.298700,0.127594,1.123788,2.330151,...,0.143134,1.333617,0.320830,1.209395,1.983253,1.801040,0.000000,0.000000,0.000000,0.000000
1,3.736162,6.597008,4.941602,1.239078,0.403210,0.318469,0.222777,0.782325,0.016133,0.016133,...,0.500033,0.977747,0.000000,0.031324,0.000000,0.000000,0.033263,0.000000,0.025164,0.000000
2,3.113306,2.854025,2.688789,1.358068,1.585247,2.266678,0.548850,0.971478,0.694944,1.597406,...,0.635072,0.196143,0.680662,0.336033,1.607502,1.960970,0.000000,0.000000,0.000000,0.005487
3,0.055429,0.000000,2.824535,0.000000,0.240313,0.059274,0.440367,0.014268,2.730922,2.545960,...,1.247056,1.347328,1.809847,0.238808,2.559675,0.465984,0.975320,1.021264,0.119180,0.146033
4,0.484349,1.256083,0.359304,1.275254,0.093606,0.042257,0.000000,0.000000,1.505058,1.778495,...,0.244099,0.451293,0.000000,0.000000,0.000000,0.000000,0.674627,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280742,1.013533,1.918724,1.680400,0.101543,0.133608,0.000000,3.744615,0.381688,3.143268,0.515954,...,0.183170,0.323274,1.467262,2.938093,0.254057,1.577386,0.024614,0.296276,0.000000,0.258263
280743,4.995702,2.675699,0.902978,2.184537,3.057896,1.699590,5.906440,2.120128,0.473211,0.476595,...,0.857434,1.365958,0.000000,0.333333,0.016225,0.801414,0.000000,0.000000,0.000000,0.000000
280744,0.000000,0.000000,0.000000,0.000000,2.423423,1.015439,1.725580,0.000000,1.052260,0.360695,...,3.084663,1.221924,0.427211,1.516287,2.285784,4.832259,0.810528,0.000000,0.040364,0.509820
280745,0.502969,0.043087,0.008109,0.000000,0.189479,0.626572,1.119680,1.533680,0.000000,0.000000,...,0.098253,0.331980,4.437345,0.875123,0.000000,0.000000,0.073967,0.000000,0.103370,0.000000


In [5]:
numpy_data = df[df.columns[2:]].to_numpy(dtype=np.float64)

In [6]:
numpy_data

array([[2.13690177e-01, 2.54143387e-01, 1.35755137e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.73616195e+00, 6.59700775e+00, 4.94160175e+00, ...,
        0.00000000e+00, 2.51641739e-02, 0.00000000e+00],
       [3.11330581e+00, 2.85402465e+00, 2.68878865e+00, ...,
        0.00000000e+00, 0.00000000e+00, 5.48664667e-03],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.03641127e-02, 5.09819686e-01],
       [5.02969325e-01, 4.30869497e-02, 8.10931623e-03, ...,
        0.00000000e+00, 1.03370100e-01, 0.00000000e+00],
       [1.43159485e+00, 3.25754666e+00, 1.78348589e+00, ...,
        0.00000000e+00, 4.61554855e-01, 9.06193316e-01]])

In [7]:
tensor_data = torch.from_numpy(numpy_data).float().to(device)
tensor_data

tensor([[2.1369e-01, 2.5414e-01, 1.3576e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.7362e+00, 6.5970e+00, 4.9416e+00,  ..., 0.0000e+00, 2.5164e-02,
         0.0000e+00],
        [3.1133e+00, 2.8540e+00, 2.6888e+00,  ..., 0.0000e+00, 0.0000e+00,
         5.4866e-03],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 4.0364e-02,
         5.0982e-01],
        [5.0297e-01, 4.3087e-02, 8.1093e-03,  ..., 0.0000e+00, 1.0337e-01,
         0.0000e+00],
        [1.4316e+00, 3.2575e+00, 1.7835e+00,  ..., 0.0000e+00, 4.6155e-01,
         9.0619e-01]], device='cuda:0')

In [8]:
tensor_data.shape

torch.Size([280747, 1024])

### 1.1 Obtaining the new encoding for test image

In [9]:
alexnet = models.alexnet(progress = True, pretrained = True)
modi_alexnet = torch.nn.Sequential(*(list(alexnet.children())[:-2]))

In [10]:
data1 = []
data2 = []
distance_loss = torch.zeros(3).to('cuda')
similarity_loss = torch.zeros(3).to('cuda')
start_time = time.time()
count = 0
for image in os.listdir('data/test'):
    input_image = Image.open('data/test/'+image)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        modi_alexnet.to('cuda')
    with torch.no_grad():
        output = modi_alexnet(input_batch)
    avg_pool = torch.nn.AdaptiveAvgPool2d(output_size=(2,2))
    output = avg_pool(output)
    output = output.flatten().reshape((1,1024))
    
    #implementing KNN using distance metric
    dist = torch.norm(tensor_data - output, dim=1, p=None)
    knn = dist.topk(3, largest=False)
    top3 = knn.indices.to('cpu').numpy()
    data1.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    distance_loss = distance_loss + knn.values
    
    #implementing KNN using cosine metric
    cosine_similarity_output = torch.cosine_similarity(tensor_data, output, dim=1, eps=1e-6)
    cso = cosine_similarity_output.topk(3, largest = True)
    top3 = cso.indices.to('cpu').numpy()
    data2.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    similarity_loss = similarity_loss + cso.values
    
    
    count = count + 1
    if(count % 500 == 0):
        print(str(count)+ '----------completed!')

df = pd.DataFrame(data1, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/alexnet/distance_knn.csv',index = False)


df = pd.DataFrame(data2, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/alexnet/similarity_knn.csv',index = False)

end = time.time()
hours, rem = divmod(end-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("total time taken = {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

500----------completed!
1000----------completed!
1500----------completed!
2000----------completed!
2500----------completed!
3000----------completed!
3500----------completed!
4000----------completed!
4500----------completed!
5000----------completed!
5500----------completed!
6000----------completed!
6500----------completed!
7000----------completed!
7500----------completed!
8000----------completed!
8500----------completed!
9000----------completed!
9500----------completed!
10000----------completed!
10500----------completed!
11000----------completed!
11500----------completed!
12000----------completed!
12500----------completed!
13000----------completed!
13500----------completed!
14000----------completed!
total time taken = 00:36:05.88


In [11]:
print(count)
avg_dist_loss = distance_loss/count
avg_similarity_loss = similarity_loss/count
avg_dist_loss, avg_similarity_loss

14236


(tensor([35.6281, 36.8806, 37.2691], device='cuda:0'),
 tensor([0.7538, 0.7388, 0.7331], device='cuda:0'))

In [12]:
similarity_loss

tensor([10730.6455, 10516.8770, 10436.0986], device='cuda:0')

In [13]:
# input_image = Image.open('data/test/000ad38272_10b735780a.jpg')
# preprocess = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])
# input_tensor = preprocess(input_image)
# input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# # move the input and model to GPU for speed if available
# if torch.cuda.is_available():
#     input_batch = input_batch.to('cuda')
#     modi_alexnet.to('cuda')
    

# with torch.no_grad():
#     output = modi_alexnet(input_batch)
# # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# avg_pool = torch.nn.AdaptiveAvgPool2d(output_size=(2,2))
# output = avg_pool(output)
# output = output.flatten().reshape((1,1024))

### 1.2 Implementing K NN based on distance metrics

In [17]:
# dist = torch.norm(tensor_data - output, dim=1, p=None)
# knn = dist.topk(3, largest=False)

# print('kNN dist: {}, index: {}'.format(knn.values, knn.indices))

kNN dist: tensor([35.4068, 35.7184, 35.9263], device='cuda:0'), index: tensor([259517, 222737,  57071], device='cuda:0')


In [26]:
# top3 = knn.indices.to('cpu').numpy()
# df[df.columns[0:2]].iloc[top3]

,id,image_id
259517,ecb0dfbe5b,2c4207316e.jpg
222737,cae6a1b7b3,af8611d549.jpg
57071,3427c61c50,1b350340d7.jpg


In [18]:
# cosine_similarity_output = torch.cosine_similarity(tensor_data, output, dim=1, eps=1e-6)
# cso = cosine_similarity_output.topk(3, largest = False)
# print('cosine similarity: {}, index: {}'.format(cso.values, cso.indices))

cosine similarity: tensor([0.7463, 0.7473, 0.7493], device='cuda:0'), index: tensor([167348, 191111, 162980], device='cuda:0')


In [28]:
# ctop3 = cso.indices.to('cpu').numpy()
# df[df.columns[0:2]].iloc[ctop3.numpy()]

,id,image_id
167348,9897ed7e4f,a0b52d4e51.jpg
191111,ae20ee73e8,43a6d9b40e.jpg
162980,94afd954cc,bbd3e7c647.jpg


## kNN on DenseNet

In [2]:
df = pd.read_csv('data/densenet_encoding/encoding_file.csv')
numpy_data = df[df.columns[2:]].to_numpy(dtype=np.float64)
tensor_data = torch.from_numpy(numpy_data).float().to(device)
del(numpy_data)

In [3]:
densenet161 = models.densenet161(pretrained=True, progress=True)
modi_densenet = torch.nn.Sequential(*(list(densenet161.children())[:-1]))

In [4]:
data1 = []
data2 = []
start_time = time.time()
count = 0
distance_loss = torch.zeros(3).to('cuda')
similarity_loss = torch.zeros(3).to('cuda')
for image in os.listdir('data/test'):
    input_image = Image.open('data/test/'+image)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        modi_densenet.to('cuda')
    with torch.no_grad():
        output = modi_densenet(input_batch)
    avg_pool = torch.nn.AdaptiveAvgPool2d((1,1))
    output = avg_pool(output)
    output = output.reshape((1,1,2208))
    avg_pool = torch.nn.AdaptiveAvgPool1d(1000)
    output = avg_pool(output)
    output = output.flatten().reshape((1,1000))
    
    #implementing KNN using distance metric
    dist = torch.norm(tensor_data - output, dim=1, p=None)
    knn = dist.topk(3, largest=False)
    top3 = knn.indices.to('cpu').numpy()
    data1.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    distance_loss = distance_loss + knn.values
    
    #implementing KNN using cosine metric
    cosine_similarity_output = torch.cosine_similarity(tensor_data, output, dim=1, eps=1e-3)
    cso = cosine_similarity_output.topk(3, largest = True)
    top3 = cso.indices.to('cpu').numpy()
    data2.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    similarity_loss = similarity_loss + cso.values
    
    count = count + 1
    if(count % 500 == 0):
        print(str(count)+ '----------completed!')

df = pd.DataFrame(data1, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/densenet/distance_knn.csv',index = False)


df = pd.DataFrame(data2, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/densenet/similarity_knn.csv',index = False)

end = time.time()
hours, rem = divmod(end-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("total time taken = {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

500----------completed!
1000----------completed!
1500----------completed!
2000----------completed!
2500----------completed!
3000----------completed!
3500----------completed!
4000----------completed!
4500----------completed!
5000----------completed!
5500----------completed!
6000----------completed!
6500----------completed!
7000----------completed!
7500----------completed!
8000----------completed!
8500----------completed!
9000----------completed!
9500----------completed!
10000----------completed!
10500----------completed!
11000----------completed!
11500----------completed!
12000----------completed!
12500----------completed!
13000----------completed!
13500----------completed!
14000----------completed!
total time taken = 00:46:33.01


In [5]:
avg_dist_loss = distance_loss/count
avg_similarity_loss = similarity_loss/count
avg_dist_loss, avg_similarity_loss

(tensor([7.9626e-07, 8.1010e-07, 8.1365e-07], device='cuda:0'),
 tensor([1.0000, 1.0000, 1.0000], device='cuda:0'))

In [6]:
similarity_loss, count

(tensor([14236.0010, 14236.0010, 14236.0010], device='cuda:0'), 14236)

## kNN on GoogleNet

In [2]:
df = pd.read_csv('data/googlenet_encoding/encoding_file.csv')
numpy_data = df[df.columns[2:]].to_numpy(dtype=np.float64)
tensor_data = torch.from_numpy(numpy_data).float().to(device)
del(numpy_data)

In [3]:
googleNet = models.googlenet(pretrained=True, progress=True)
modi_googleNet = torch.nn.Sequential(*(list(googleNet.children())[:-1]))

In [4]:
data1 = []
data2 = []
start_time = time.time()
count = 0
distance_loss = torch.zeros(3).to('cuda')
similarity_loss = torch.zeros(3).to('cuda')

for image in os.listdir('data/test'):
    input_image = Image.open('data/test/'+image)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        modi_googleNet.to('cuda')
    with torch.no_grad():
        output = modi_googleNet(input_batch)
    output = output.flatten().reshape((1,1024))
    
    #implementing KNN using distance metric
    dist = torch.norm(tensor_data - output, dim=1, p=None)
    knn = dist.topk(3, largest=False)
    top3 = knn.indices.to('cpu').numpy()
    data1.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    distance_loss = distance_loss + knn.values
    
    #implementing KNN using cosine metric
    cosine_similarity_output = torch.cosine_similarity(tensor_data, output, dim=1, eps=1e-6)
    cso = cosine_similarity_output.topk(3, largest = True)
    top3 = cso.indices.to('cpu').numpy()
    data2.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    similarity_loss = similarity_loss + cso.values
    
    count = count + 1
    if(count % 500 == 0):
        print(str(count)+ '----------completed!')

df = pd.DataFrame(data1, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/googlenet/distance_knn.csv',index = False)


df = pd.DataFrame(data2, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/googlenet/similarity_knn.csv',index = False)

end = time.time()
hours, rem = divmod(end-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("total time taken = {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

500----------completed!
1000----------completed!
1500----------completed!
2000----------completed!
2500----------completed!
3000----------completed!
3500----------completed!
4000----------completed!
4500----------completed!
5000----------completed!
5500----------completed!
6000----------completed!
6500----------completed!
7000----------completed!
7500----------completed!
8000----------completed!
8500----------completed!
9000----------completed!
9500----------completed!
10000----------completed!
10500----------completed!
11000----------completed!
11500----------completed!
12000----------completed!
12500----------completed!
13000----------completed!
13500----------completed!
14000----------completed!
total time taken = 00:38:04.81


In [5]:
avg_dist_loss = distance_loss/count
avg_similarity_loss = similarity_loss/count
avg_dist_loss, avg_similarity_loss

(tensor([7.8559, 7.8938, 7.9132], device='cuda:0'),
 tensor([0.8374, 0.8357, 0.8349], device='cuda:0'))

## kNN on ResNet

In [2]:
df = pd.read_csv('data/resnet_encoding/encoding_file.csv')
numpy_data = df[df.columns[2:]].to_numpy(dtype=np.float64)
tensor_data = torch.from_numpy(numpy_data).float().to(device)
del(numpy_data)

In [3]:
resnet152 = models.resnet152(pretrained=True, progress=True)
modi_resnet = torch.nn.Sequential(*(list(resnet152.children())[:-1]))

In [4]:
data1 = []
data2 = []
start_time = time.time()
count = 0
distance_loss = torch.zeros(3).to('cuda')
similarity_loss = torch.zeros(3).to('cuda')

for image in os.listdir('data/test'):
    input_image = Image.open('data/test/'+image)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        modi_resnet.to('cuda')
    with torch.no_grad():
        output = modi_resnet(input_batch)
    output = output.reshape((1,1,2048))
    avg_pool = torch.nn.AdaptiveAvgPool1d(output_size=(1000))
    output = avg_pool(output)
    output = output.flatten().reshape((1,1000))
    
    #implementing KNN using distance metric
    dist = torch.norm(tensor_data - output, dim=1, p=None)
    knn = dist.topk(3, largest=False)
    top3 = knn.indices.to('cpu').numpy()
    data1.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    distance_loss = distance_loss + knn.values
    
    #implementing KNN using cosine metric
    cosine_similarity_output = torch.cosine_similarity(tensor_data, output, dim=1, eps=1e-6)
    cso = cosine_similarity_output.topk(3, largest = True)
    top3 = cso.indices.to('cpu').numpy()
    data2.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    similarity_loss = similarity_loss + cso.values
    
    count = count + 1
    if(count % 500 == 0):
        print(str(count)+ '----------completed!')

df = pd.DataFrame(data1, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/resnet/distance_knn.csv',index = False)


df = pd.DataFrame(data2, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/resnet/similarity_knn.csv',index = False)

end = time.time()
hours, rem = divmod(end-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("total time taken = {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

500----------completed!
1000----------completed!
1500----------completed!
2000----------completed!
2500----------completed!
3000----------completed!
3500----------completed!
4000----------completed!
4500----------completed!
5000----------completed!
5500----------completed!
6000----------completed!
6500----------completed!
7000----------completed!
7500----------completed!
8000----------completed!
8500----------completed!
9000----------completed!
9500----------completed!
10000----------completed!
10500----------completed!
11000----------completed!
11500----------completed!
12000----------completed!
12500----------completed!
13000----------completed!
13500----------completed!
14000----------completed!
total time taken = 00:46:21.41


In [5]:
avg_dist_loss = distance_loss/count
avg_similarity_loss = similarity_loss/count
avg_dist_loss, avg_similarity_loss

(tensor([0.7635, 0.7755, 0.7779], device='cuda:0'),
 tensor([0.9988, 0.9988, 0.9988], device='cuda:0'))

## kNN on ResNext

In [3]:
df = pd.read_csv('data/resnext_encoding/encoding_file.csv')
numpy_data = df[df.columns[2:]].to_numpy(dtype=np.float64)
tensor_data = torch.from_numpy(numpy_data).float().to(device)
del(numpy_data)

In [4]:
resnext = torch.hub.load('pytorch/vision:v0.6.0', 'resnext101_32x8d', pretrained=True)
modi_resnext = torch.nn.Sequential(*(list(resnext.children())[:-1]))

Using cache found in C:\Users\balag/.cache\torch\hub\pytorch_vision_v0.6.0


In [5]:
data1 = []
data2 = []
start_time = time.time()
count = 0
distance_loss = torch.zeros(3).to('cuda')
similarity_loss = torch.zeros(3).to('cuda')

for image in os.listdir('data/test'):
    input_image = Image.open('data/test/'+image)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        modi_resnext.to('cuda')
    with torch.no_grad():
        output = modi_resnext(input_batch)
    output = output.reshape((1,1,2048))
    avg_pool = torch.nn.AdaptiveAvgPool1d(output_size=(1000))
    output = avg_pool(output)
    output = output.flatten().reshape((1,1000))
    
    #implementing KNN using distance metric
    dist = torch.norm(tensor_data - output, dim=1, p=None)
    knn = dist.topk(3, largest=False)
    top3 = knn.indices.to('cpu').numpy()
    data1.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    distance_loss = distance_loss + knn.values
    
    #implementing KNN using cosine metric
    cosine_similarity_output = torch.cosine_similarity(tensor_data, output, dim=1, eps=1e-6)
    cso = cosine_similarity_output.topk(3, largest = True)
    top3 = cso.indices.to('cpu').numpy()
    data2.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    similarity_loss = similarity_loss + cso.values
    
    count = count + 1
    if(count % 100 == 0):
        print(str(count)+ '----------completed!')

df = pd.DataFrame(data1, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/resnext/distance_knn.csv',index = False)


df = pd.DataFrame(data2, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/resnext/similarity_knn.csv',index = False)

end = time.time()
hours, rem = divmod(end-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("total time taken = {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

100----------completed!
200----------completed!
300----------completed!
400----------completed!
500----------completed!
600----------completed!
700----------completed!
800----------completed!
900----------completed!
1000----------completed!
1100----------completed!
1200----------completed!
1300----------completed!
1400----------completed!
1500----------completed!
1600----------completed!
1700----------completed!
1800----------completed!
1900----------completed!
2000----------completed!
2100----------completed!
2200----------completed!
2300----------completed!
2400----------completed!
2500----------completed!
2600----------completed!
2700----------completed!
2800----------completed!
2900----------completed!
3000----------completed!
3100----------completed!
3200----------completed!
3300----------completed!
3400----------completed!
3500----------completed!
3600----------completed!
3700----------completed!
3800----------completed!
3900----------completed!
4000----------completed!
4100-----

In [6]:
avg_dist_loss = distance_loss/count
avg_similarity_loss = similarity_loss/count
avg_dist_loss, avg_similarity_loss

(tensor([0.7643, 0.7780, 0.7813], device='cuda:0'),
 tensor([0.9986, 0.9986, 0.9986], device='cuda:0'))

## kNN on VGG

In [2]:
df = pd.read_csv('data/vgg_encoding/encoding_file.csv')
numpy_data = df[df.columns[2:]].to_numpy(dtype=np.float64)
tensor_data = torch.from_numpy(numpy_data).float().to(device)
del(numpy_data)

In [3]:
vgg19 = models.vgg19_bn(pretrained=True, progress=True)
modi_vgg = torch.nn.Sequential(*(list(vgg19.children())[:-2]))

In [4]:
data1 = []
data2 = []
start_time = time.time()
count = 0
distance_loss = torch.zeros(3).to('cuda')
similarity_loss = torch.zeros(3).to('cuda')

for image in os.listdir('data/test'):
    input_image = Image.open('data/test/'+image)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        modi_vgg.to('cuda')
    with torch.no_grad():
        output = modi_vgg(input_batch)
    avg_pool = torch.nn.AdaptiveAvgPool2d((2,1))
    output = avg_pool(output)
    output = output.flatten().reshape((1,1024))
    
    #implementing KNN using distance metric
    dist = torch.norm(tensor_data - output, dim=1, p=None)
    knn = dist.topk(3, largest=False)
    top3 = knn.indices.to('cpu').numpy()
    data1.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    distance_loss = distance_loss + knn.values
    
    #implementing KNN using cosine metric
    cosine_similarity_output = torch.cosine_similarity(tensor_data, output, dim=1, eps=1e-6)
    cso = cosine_similarity_output.topk(3, largest = True)
    top3 = cso.indices.to('cpu').numpy()
    data2.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    similarity_loss = similarity_loss + cso.values
    
    count = count + 1
    if(count % 500 == 0):
        print(str(count)+ '----------completed!')

df = pd.DataFrame(data1, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/vgg/distance_knn.csv',index = False)


df = pd.DataFrame(data2, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/vgg/similarity_knn.csv',index = False)

end = time.time()
hours, rem = divmod(end-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("total time taken = {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

500----------completed!
1000----------completed!
1500----------completed!
2000----------completed!
2500----------completed!
3000----------completed!
3500----------completed!
4000----------completed!
4500----------completed!
5000----------completed!
5500----------completed!
6000----------completed!
6500----------completed!
7000----------completed!
7500----------completed!
8000----------completed!
8500----------completed!
9000----------completed!
9500----------completed!
10000----------completed!
10500----------completed!
11000----------completed!
11500----------completed!
12000----------completed!
12500----------completed!
13000----------completed!
13500----------completed!
14000----------completed!
total time taken = 00:40:09.28


In [5]:
avg_dist_loss = distance_loss/count
avg_similarity_loss = similarity_loss/count
avg_dist_loss, avg_similarity_loss

(tensor([3.1642, 3.2286, 3.2448], device='cuda:0'),
 tensor([0.9034, 0.9004, 0.8995], device='cuda:0'))

In [2]:
df = pd.read_csv('data/resnext_encoding/encoding_file_1.csv')
numpy_data = df[df.columns[2:]].to_numpy(dtype=np.float64)
tensor_data = torch.from_numpy(numpy_data).float()
del(numpy_data)

In [3]:
resnext = torch.hub.load('pytorch/vision:v0.6.0', 'resnext101_32x8d', pretrained=True)
modi_resnext = torch.nn.Sequential(*(list(resnext.children())[:-1]))

Using cache found in C:\Users\balag/.cache\torch\hub\pytorch_vision_v0.6.0


In [ ]:
data1 = []
data2 = []
start_time = time.time()
count = 0
distance_loss = torch.zeros(3)
similarity_loss = torch.zeros(3)

for image in os.listdir('data/test'):
    input_image = Image.open('data/test/'+image)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
    # move the input and model to GPU for speed if available
#     if torch.cuda.is_available():
#         input_batch = input_batch.to('cuda')
#         modi_resnext.to('cuda')
    with torch.no_grad():
        output = modi_resnext(input_batch)
#     output = output.reshape((1,1,2048))
#     avg_pool = torch.nn.AdaptiveAvgPool1d(output_size=(1000))
#     output = avg_pool(output)
    output = output.flatten().reshape((1,2048))
    
    #implementing KNN using distance metric
    dist = torch.norm(tensor_data - output, dim=1, p=None)
    knn = dist.topk(3, largest=False)
    top3 = knn.indices.numpy()
    data1.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    distance_loss = distance_loss + knn.values
    
    #implementing KNN using cosine metric
    cosine_similarity_output = torch.cosine_similarity(tensor_data, output, dim=1, eps=1e-6)
    cso = cosine_similarity_output.topk(3, largest = True)
    top3 = cso.indices.numpy()
    data2.append([image[0:10], image[11:],
                  df['id'].iloc[top3[0]], df['image_id'].iloc[top3[0]], 
                  df['id'].iloc[top3[1]], df['image_id'].iloc[top3[1]], 
                  df['id'].iloc[top3[2]], df['image_id'].iloc[top3[2]]])
    
    similarity_loss = similarity_loss + cso.values
    
    count = count + 1
    if(count % 100 == 0):
        print(str(count)+ '----------completed!')

df = pd.DataFrame(data1, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/resnext/distance_knn2.csv',index = False)


df = pd.DataFrame(data2, columns = ['test_id', 'test_image_id',
                                    'top1_id', 'top1_image_id',
                                    'top2_id', 'top2_image_id',
                                    'top3_id', 'top3_image_id',])
df.to_csv('data/output/resnext/similarity_knn2.csv',index = False)

end = time.time()
hours, rem = divmod(end-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("total time taken = {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

100----------completed!
200----------completed!
300----------completed!
400----------completed!
500----------completed!
600----------completed!
700----------completed!
800----------completed!
900----------completed!
1000----------completed!
1100----------completed!
1200----------completed!
1300----------completed!
1400----------completed!
1500----------completed!
1600----------completed!
1700----------completed!
1800----------completed!
1900----------completed!
2000----------completed!
2100----------completed!
2200----------completed!
2300----------completed!
2400----------completed!
2500----------completed!
2600----------completed!
2700----------completed!
2800----------completed!
2900----------completed!
3000----------completed!
3100----------completed!
3200----------completed!
3300----------completed!
3400----------completed!
3500----------completed!
3600----------completed!
3700----------completed!
3800----------completed!
3900----------completed!
4000----------completed!
4100-----

In [6]:
avg_dist_loss = distance_loss/count
avg_similarity_loss = similarity_loss/count
avg_dist_loss, avg_similarity_loss

(tensor([3.1642, 3.2286, 3.2448], device='cuda:0'),
 tensor([0.9034, 0.9004, 0.8995], device='cuda:0'))